In [76]:
## Cell 1 – Importy i konfiguracja

import os
import cv2
import numpy as np

# Do manipulacji kolorem (saturacja).
import colorsys

# Instalowanie bibliotek potrzebnych do dzialania.
# !pip install opencv-python

In [77]:
## Cell 2 – Sciezki do oryginalnych plikow i folder wyjsciowy

# Lista sciezek do plikow zrodlowych (jpg / png / bmp itp.), ktore maja byc zaburzone.
image_paths = [
    r"C:\Users\MrNotYourBusiness\Desktop\Image-similarity-metrics\Photos_to_distortions\Degus.bmp",
    r"C:\Users\MrNotYourBusiness\Desktop\Image-similarity-metrics\Photos_to_distortions\Hotdog.bmp",
    r"C:\Users\MrNotYourBusiness\Desktop\Image-similarity-metrics\Photos_to_distortions\Ziemniaki.bmp"
    # 'ścieżka/do/obrazu.png',
]

# Tworzenie folderu z wynikami.
output_root = 'Distortions_v2'

In [78]:
## Cell 3 – Ustawienia wartosci poziomow zaburzen

# Dla każdej metody podajemy liste 4 parametrow (poziomy 1–4).
# Poziom "0" oznacza – brak zaburzenia (oryginal).
# Zmiana wartosci spowoduje, ze przy kolejnych uruchomieniu zostana uzyte nowe wartosci.

config = {
    'blur': {             # sigma dla Gaussowskiego rozmycia
        'levels': [1.0, 2.0, 5.0, 10.0, 1000.0]
    },
    'brightness': {       # mnożnik jasności (1 = oryginał)
        'levels': [1, 2, 5, 20]
    },
    'circular_blur': {    # promień (w px) filtra "okrągłego" uśredniającego
        'levels': [3, 5, 7, 9]
    },
    'dilation': {         # promień (w px) strukturalnego elementu elipsy
        'levels': [3, 5, 7, 9]
    },
    'hole': {             # ułamek min(wysokość, szerokość) decyduje o promieniu w px
        'levels': [0.05, 0.1, 0.15, 0.2]
    },
    'noise': {            # odchylenie standardowe Gaussowskiego szumu
        'levels': [0.01, 0.05, 0.1, 0.25]
    },
    'rotate': {           # kąt w stopniach
        'levels': [1, 2, 5, 10]
    },
    'saturation': {       # mnożnik nasycenia (1 = oryginał)
        'levels': [1.1, 1.5, 2, 5]
    },
    'shift': {            # przesunięcie w pikselach (dx, dy) – używamy (lvl,lvl)
        'levels': [1, 2, 5, 10]
    }
}


In [79]:
## Cell 4 – Pomocniczne funkcje do kazdego rodzaju zaburzenia

def apply_blur(img, sigma):
    # GaussianBlur: jeżeli kernel=(0,0), to sigmaX = sigma
    return cv2.GaussianBlur(img, (0, 0), sigmaX=sigma)

def apply_brightness(img, factor):
    # img w formacie BGR, uint8
    tmp = img.astype(np.float32) * factor
    tmp = np.clip(tmp, 0, 255).astype(np.uint8)
    return tmp

def apply_circular_blur(img, radius):
    # Tworzymy macierz o wymiarach (2r+1) × (2r+1) z wartościami 1 w obrębie koła
    k = 2 * radius + 1
    kernel = np.zeros((k, k), dtype=np.float32)
    cx, cy = radius, radius
    for i in range(k):
        for j in range(k):
            if (i - cy) ** 2 + (j - cx) ** 2 <= radius**2:
                kernel[i, j] = 1.0
    # Normalizacja
    s = np.sum(kernel)
    if s > 0:
        kernel /= s
    # Filtrujemy obraz dla każdego kanału
    blurred = cv2.filter2D(img, -1, kernel)
    return blurred

def apply_dilation(img, kernel_radius):
    # Element eliptyczny o rozmiarze (2r+1)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*kernel_radius+1, 2*kernel_radius+1))
    return cv2.dilate(img, k)

def apply_hole(img, frac):
    # Rysujemy czarne kółko na środku obrazu o promieniu = frac × min(h, w)
    h, w = img.shape[:2]
    radius = int(min(h, w) * frac)
    out = img.copy()
    center = (w // 2, h // 2)
    cv2.circle(out, center, radius, (0, 0, 0), thickness=-1)
    return out

def apply_noise(img, stddev):
    # Gaussowski szum dla każdego kanału BGR
    noise = np.random.normal(0, stddev, img.shape).astype(np.float32)
    tmp = img.astype(np.float32) + noise
    tmp = np.clip(tmp, 0, 255).astype(np.uint8)
    return tmp

def apply_rotate(img, angle):
    h, w = img.shape[:2]
    center = (w / 2, h / 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    # nowy rozmiar = oryginalny (wypełniamy czarnym tłem)
    rotated = cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
    return rotated

def apply_saturation(img, factor):
    # Konwersja BGR -> HSV, mnożymy kanał S, konwersja z powrotem
    # cv2 używa skali H:0–179, S:0–255, V:0–255
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype(np.float32)
    hsv[:, :, 1] *= factor
    hsv[:, :, 1] = np.clip(hsv[:, :, 1], 0, 255)
    out = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)
    return out

def apply_shift(img, shift_px):
    h, w = img.shape[:2]
    dx = shift_px
    dy = shift_px
    M = np.array([[1, 0, dx], [0, 1, dy]], dtype=np.float32)
    shifted = cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
    return shifted


In [80]:
## Cell 5 – Mapa nazwa_metody -> funkcja

# Mapowanie nazwy zaburzenia (klucz ze slownika config) na funkcje, ktora je wykonuje
func_map = {
    'blur': apply_blur,
    'brightness': apply_brightness,
    'circular_blur': apply_circular_blur,
    'dilation': apply_dilation,
    'hole': apply_hole,
    'noise': apply_noise,
    'rotate': apply_rotate,
    'saturation': apply_saturation,
    'shift': apply_shift
}


In [81]:
## Cell 6 – Glowna petla tworzaca foldery i zapisujaca pliki

for img_path in image_paths:
    # 1) Wczytanie oryginalnego obrazu
    img = cv2.imread(img_path)
    if img is None:
        print(f'⚠ Nie udało się wczytać {img_path}. Pomijasz ten plik.')
        continue

    # 2) Nazwa bazowa bez rozszerzenia i sciezka docelowego folderu
    basename = os.path.splitext(os.path.basename(img_path))[0]
    base_out = os.path.join(output_root, basename)
    ext = os.path.splitext(img_path)[1]  

    # 3) Tworzymy katalog wynikowy dla tego obrazu (jezeli nie istnieje)
    os.makedirs(base_out, exist_ok=True)

    # 4) Dla każdej metody zaburzenia:
    for method, params in config.items():
        levels = params['levels']   # lista 4 wartosci
        f = func_map[method]       # funkcja zaburzenia

        # Katalog subfolderu np. Distortions_v2/zdjecie/blur
        subfolder = os.path.join(base_out, method.capitalize())
        os.makedirs(subfolder, exist_ok=True)

        # 4a) Poziom 0: zapisz oryginał (bez zaburzeń), nazwa pliku: "<basename>_<method>_0.png"
        out0 = os.path.join(subfolder, f"{basename}_{method}_0{ext}")
        if not os.path.exists(out0):
            cv2.imwrite(out0, img)
        # jeśli już istnieje, pomijamy i nie nadpisujemy

        # 4b) Poziomy 1–4: iterujemy po wartościach levels
        for val in levels:
            # w nazwie pliku ma być 'wartość parametru' – np. "2", "0.75", "30"...
            # Więc wykorzystamy dokładnie tę liczbę z config
            str_val = str(val).replace('.', '_')  # zamieniamy '.' -> '_' żeby uniknąć problemów z nazwą
            out_path = os.path.join(subfolder, f"{basename}_{method}_{str_val}.png")

            if os.path.exists(out_path):
                # już jest taki plik, pomijamy
                continue

            # 4c) Tworzymy obraz z zaburzeniem
            try:
                disturbed = f(img, val)
                cv2.imwrite(out_path, disturbed)
            except Exception as e:
                print(f"Błąd przy {method}={val} dla {img_path}: {e}")


KeyboardInterrupt: 

In [ ]:
## Cell 7 – Informacja o zakończeniu

print("Generowanie zaburzen zakonczone.")
print(f"Sprawdź katalog '{output_root}' – tam znajdziesz foldery dla każdego oryginału i podfoldery z zaburzeniami.")
